In [2]:
import re
import matplotlib.pyplot as plt
import numpy as np
import os
%matplotlib inline
# with serifs -> looks less out of place when used in latex files
# plt.rcParams["font.family"] = "serif"
write_pdf = False
# without serifs -> better for general viewing or for powerpoint
plt.rcParams["font.family"] = "sans-serif"


In [ ]:
experiment_header_regex = re.compile(r"Experiment: [a-z|_]+ [a-z]+$")
header_regex = re.compile(r"Experiment: (.*) num_narrow_bursts: (\d+) " + \
    r"num_wide_bursts: (\d+) wide_burst_length: (\d+)")
data_regex = re.compile(r"(Old router|VC router ): \[latency \+- std, BW \+- std\]: " + \
    r"narrow: \[(\d+\.\d+) \+- (\d+\.\d+), (\d+\.\d+) \+- (\d+\.\d+)\], " + \
    r"wide: \[(\d+\.\d+) \+- (\d+\.\d+), (\d+\.\d+) \+- (\d+\.\d+)\]")
traffic_types_ordered = ['random read', 'random write', 'bit_complement read', 'bit_complement write', 'bit_reverse read', 'bit_reverse write', 'bit_rotation read', 'bit_rotation write', 'shuffle read', 'shuffle write', 'transpose read', 'transpose write', 'neighbor read', 'neighbor write', 'tornado read', 'tornado write', 'onehop read', 'onehop write', 'single_dest_center read', 'single_dest_center write', 'single_dest_boundary read', 'single_dest_boundary write', 'hbm read', 'hbm write', 'average (geo mean)']

def get_data(experiment_name):
    traffic_types = []
    parameter_combinations = []
    traffic_type = 0
    parameter_combination_index = 0
    data_list = []
    errors = 0
    with open(os.getcwd() + f"/util/output/dma_experiments_{experiment_name}.txt", "r") as file:
        for line in file:
            line = line.strip()
            if line == "":
                continue
            experiment_header_match = experiment_header_regex.match(line)
            header_match = header_regex.match(line)
            data_match = data_regex.match(line)
            if experiment_header_match:
                continue
            elif header_match:
                header_match = header_match.groups()
                if header_match[0] not in traffic_types:
                    traffic_types.append(header_match[0])
                traffic_type = header_match[0]
                if (int(header_match[1]), int(header_match[2]), int(header_match[3])) not in parameter_combinations:
                    parameter_combinations.append((int(header_match[1]), int(header_match[2]), int(header_match[3])))
                parameter_combination_index = parameter_combinations.index((int(header_match[1]), int(header_match[2]), int(header_match[3])))
            elif data_match:
                data_match = data_match.groups()
                data_list.append((traffic_type, parameter_combination_index, data_match[0], data_match[1], data_match[2], data_match[3], data_match[4], data_match[5], data_match[6], data_match[7], data_match[8]))
            else:
                print("Error: line not matched: ", line)
                errors+=1
                if(errors > 5):
                    break
    traffic_types.append("average (geo mean)")
    parameter_combinations.append("average (geo mean)")

    if(set(traffic_types) == set(traffic_types_ordered)): # set order if got all traffic types
        traffic_types = traffic_types_ordered
    data_array = np.ndarray((len(traffic_types), len(parameter_combinations), 2, 2), dtype=float)
    data_array.fill(0.0)
    for data in data_list:
        assert(data[2] in ("VC router ", "Old router"))
        index = 0 if data[2] == "Old router" else 1
        traffic_type_index = traffic_types.index(data[0])
        # narrow
        data_array[traffic_type_index][data[1]][index][0] += float(data[3]) # latency
        # data_array[data[0]][data[1]][index][1] = float(data[4]) # latency std
        # data_array[data[0]][data[1]][index][3] = float(data[5]) # bandwidth
        # data_array[data[0]][data[1]][index][3] = float(data[6]) # bandwidth std
        # wide
        # data_array[data[0]][data[1]][index][4] = float(data[7]) # latency
        # data_array[data[0]][data[1]][index][5] = float(data[8]) # latency std
        data_array[traffic_type_index][data[1]][index][1] += float(data[9]) # bandwidth
        # data_array[data[0]][data[1]][index][7] = float(data[10])# bandwidth std
    if("random read" in traffic_types):
        data_array[traffic_types.index("random read"),:,:,:] /= 5
    if("random write" in traffic_types):
        data_array[traffic_types.index("random write"),:,:,:] /= 5
    data_array[-1,:,:,:] = np.prod(data_array[0:-1,:,:,:], axis=0)**(1.0/(len(traffic_types)-1))
    data_array[:,-1,:,:] = np.prod(data_array[:,0:-1,:,:], axis=1)**(1.0/(len(parameter_combinations)-1))
    return data_array, traffic_types, parameter_combinations

In [ ]:
def plot_experiment(data_array, traffic_types, parameter_combinations, experiment_name, bandwidth):
    narrow_latency_comparison = 100*(data_array[:,:,1,0] - data_array[:,:,0,0]) / data_array[:,:,0,0]
    wide_bw_comparison = 100*(data_array[:,:,1,1] - data_array[:,:,0,1]) / data_array[:,:,0,1]
    plt.rcParams['figure.figsize'] = [10, 5]
    fig, ax = plt.subplots()
    # negate since color looks better if green is good
    data = wide_bw_comparison if bandwidth else narrow_latency_comparison
    ax.imshow(np.transpose(data if bandwidth else -data), cmap="RdYlGn", vmin=-10, vmax=10)

    # Show all ticks and label them with the respective list entries
    traffic_types = list(map(lambda x: x.replace("_", " "), traffic_types))
    ax.set_xticks(np.arange(len(traffic_types)), labels=traffic_types)
    ax.set_yticks(np.arange(len(parameter_combinations)), labels=parameter_combinations)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
            rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    for i in range(len(parameter_combinations)):
        for j in range(len(traffic_types)):
            ax.text(j, i, f"{data[j, i]:.1f}",
                ha="center", va="center", color="black", fontsize=7)
    explanation = "Wide Bandwidth: $100*(BW_{vc}-BW_{old})\ /\ BW_{old}$" if bandwidth \
      else "Narrow Latency: $100*(Lat_{vc}-Lat_{old})\ /\ Lat_{old}$"
    if(experiment_name == "SingleStageOgRouter"):
        explanation = "Wide Bandwidth: $100*(BW_{single}-BW_{two})\ /\ BW_{two}$" if bandwidth \
            else "Narrow Latency: $100*(Lat_{single}-Lat_{two})\ /\ Lat_{two}$"
        ax.set_title(f"Single vs two stage original router: {explanation}")
    else:
        ax.set_title(f"{experiment_name}: {explanation}")
    fig.set_size_inches(8, 8)
    bw_string = "bw" if bandwidth else "lat"
    if write_pdf:
        plt.savefig(f"util/plots/{experiment_name}_{bw_string}.pdf", bbox_inches='tight')
    plt.show()


def analyze_experiment(experiment_name, bandwidth = False, slow = False):
    experiment_addon = "slowSlave" if slow else ""
    try:
        data, traffic_types, parameter_combinations = get_data(experiment_addon + experiment_name)
        plot_experiment(data, traffic_types, parameter_combinations, experiment_addon + experiment_name, bandwidth)
    except FileNotFoundError:
        print(f"File not found: output/dma_experiments_{experiment_addon + experiment_name}.txt")

no_overflow_experiments = ("VCd3NoOverflow", "VCd3NoOverflowVCL1", "VCd2NoOverflow", "VCd2NoOverflowVCL1")
single_vc_experiments = ("SingleVCd3","SingleVCd2")
vc_depth_experiments = ("VCd2", "VCd3", "VCd2VCL1", "VCd3VCL1")
whd3_experiments = ("VCd2Whd3", "VCd2Whd3VCL1", "VCd2Whd3NoOverflowFromWhVC", "VCd2Whd3VCL1NoOverflowFromWhVC")
no_credit_shortcut_experiments = ("VCd2", "VCd2NoCreditShortcut", "VCd2NoOverflow", "VCd2NoOverflowNoCreditShortcut","VCd3","VCd3NoCreditShortcut", "VCd3NoOverflow","VCd3NoOverflowNoCreditShortcut")
all_experiments = no_overflow_experiments + single_vc_experiments + vc_depth_experiments + whd3_experiments + no_credit_shortcut_experiments
all_experiments = sorted(tuple(set(all_experiments)))
# not part of all experiments
single_stage_experiments = ("SingleStageOgRouter", "SingleStageSingleVCd2", "SingleStageSingleVCd3", "SingleStageVCd2VCL1", "SingleStageVCd2VCL1NoOverflow", "SingleStageVCd2Whd3VCL1", "SingleStageVCd3VCL1", "SingleStageVCd3VCL1NoOverflow" )

In [ ]:
# slow = 0      # 0 for fast, 1 for slow slave
# bandwidth = 1 # 0 for latency, 1 for bandwidth
# for experiment in all_experiments:
#      analyze_experiment(experiment, bandwidth, slow)
for experiment in list(all_experiments) + list(single_stage_experiments):
     analyze_experiment(experiment, 1, 0)
     analyze_experiment(experiment, 0, 0)
     analyze_experiment(experiment, 1, 1)
     analyze_experiment(experiment, 0, 1)

In [ ]:
def plot_parameter_combination(parameter_combination, experiment_names_og, parameter_combination_explanation="", bandwidth=1, slow=0):
  """parameter combination can also be a list of parameter combinations, then it will plot geomean of them"""
  experiment_names = list(experiment_names_og.copy())
  experiment_addon = "slowSlave" if slow else ""
  experiment_names= list(map(lambda x: experiment_addon + x, experiment_names))
  relevant_data = []
  traffic_types = None

  #get data
  for experiment_name_og in experiment_names_og:
    experiment_name = experiment_addon + experiment_name_og
    try:
      data_exp, traffic_types_exp, parameter_combinations_exp = get_data(experiment_name)
    except FileNotFoundError:
        print(f"File not found: output/dma_experiments_{experiment_name}.txt")
        experiment_names.remove(experiment_name)
        continue
    if traffic_types is None:
      traffic_types = traffic_types_exp
    else: # check same traffic types exist
      if traffic_types != traffic_types_exp:
        print(f"Traffic types in {experiment_name} do not match previous experiments")
        experiment_names.remove(experiment_name)
        continue
    if isinstance(parameter_combination, list):
      not_all_p_found = False
      for p in parameter_combination:
        if p not in parameter_combinations_exp:
          print(f"Parameter combination {p} not found in {experiment_name}")
          experiment_names.remove(experiment_name)
          not_all_p_found = True
          break
      if not_all_p_found:
        continue
      relevant_data.append(
        np.prod([data_exp[:,parameter_combinations_exp.index(p),:,:] for p in parameter_combination], axis=0)
          **(1.0/len(parameter_combination)))
        #  np.sum([data_exp[:,parameter_combinations_exp.index(p),:,:] for p in parameter_combination], axis=0))
    else:
      if parameter_combination in parameter_combinations_exp:
        relevant_data.append(data_exp[:,parameter_combinations_exp.index(parameter_combination),:,:])
      else:
        print(f"Parameter combination {parameter_combination} not found in {experiment_name}")
        experiment_names.remove(experiment_name)
  relevant_data = np.transpose(np.array(relevant_data), axes=(1,0,2,3))
  #plot data
  narrow_latency_comparison = 100*(relevant_data[:,:,1,0] - relevant_data[:,:,0,0]) / relevant_data[:,:,0,0]
  wide_bw_comparison = 100*(relevant_data[:,:,1,1] - relevant_data[:,:,0,1]) / relevant_data[:,:,0,1]
  plt.rcParams['figure.figsize'] = [10, 5]
  fig, ax = plt.subplots()
  # negate since color looks better if green is good
  data = wide_bw_comparison if bandwidth else narrow_latency_comparison
  ax.imshow(np.transpose(data if bandwidth else -data), cmap="RdYlGn", vmin=-10, vmax=10)
  if "SingleStageOgRouter" in experiment_names:
    experiment_names[experiment_names.index("SingleStageOgRouter")] = "Single vs two stage original router"

  # Show all ticks and label them with the respective list entries
  traffic_types = list(map(lambda x: x.replace("_", " "), traffic_types))
  ax.set_xticks(np.arange(len(traffic_types)), labels=traffic_types)
  ax.set_yticks(np.arange(len(experiment_names)), labels=experiment_names)

  # Rotate the tick labels and set their alignment.
  plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
          rotation_mode="anchor")

  # Loop over data dimensions and create text annotations.
  for i in range(len(experiment_names)):
      for j in range(len(traffic_types)):
          ax.text(j, i, f"{data[j, i]:.1f}",
              ha="center", va="center", color="black", fontsize=7)
  explanation = "Wide Bandwidth: $100*(bw_{vc}-bw_{old})\ /\ bw_{old}$" if bandwidth \
      else "Narrow Latency: $100*(Lat_{vc}-Lat_{old})\ /\ Lat_{old}$"

  ax.set_title(f"{parameter_combination_explanation}: {explanation}")
  fig.set_size_inches(8, 8)
  bw_string = "bw" if bandwidth else "lat"
  if write_pdf:
      plt.savefig(f"util/plots/{parameter_combination_explanation.lower().replace(',', '').replace(' ', '_')}_{bw_string}.pdf", bbox_inches='tight')
  plt.show()

def plot_all_parameter_combinations(bandwidth = 1, slow = 0, single_stage=0):
  all_parameter_combinations = ((4,4,1), (4,4,16), (4,16,1), (4,16,16), (16,16,4), (16,16,8), (16,16,16), (16,32,16), (16,64,16), (32,16,16), (64,16,16), (64,1,4), (64,4,4), (64,16,4), (1,64,4), (4,64,4), (16,64,4), (32,64,4))
  slow_str = "Slow Slave " if slow else ""
  for p in all_parameter_combinations:
    parameter_combination_explanation = "Low Load " if p == (4,4,1) else \
    "Low Load, Bursts " if p == (4,4,16) else ""
    parameter_combination_explanation += str(p)
    if single_stage:
      plot_parameter_combination(p, list(single_stage_experiments),slow_str + "Single Stage " + parameter_combination_explanation, bandwidth, slow)
    else:
      plot_parameter_combination(p, all_experiments,slow_str + parameter_combination_explanation, bandwidth, slow)

def plot_geomean_of_high_load(bandwidth=1, slow=0, single_stage=0):
  high_load_parameter_combinations =  list(((16,16,16), (16,32,16), (16,64,16), (32,16,16), (64,16,16)))
  slow_str = "Slow Slave " if slow else ""

  if single_stage:
    plot_parameter_combination(high_load_parameter_combinations, list(single_stage_experiments), slow_str + "Single Stage High Load", bandwidth, slow)
  else:
    plot_parameter_combination(high_load_parameter_combinations, all_experiments, slow_str + "High Load", bandwidth, slow)

def plot_geomean_of_all(bandwidth=1, slow=0, single_stage=0):
  all_parameter_combinations = list(((4,4,1), (4,4,16), (4,16,1), (4,16,16), (16,16,4), (16,16,8), (16,16,16), (16,32,16), (16,64,16), (32,16,16), (64,16,16), (64,1,4), (64,4,4), (64,16,4), (1,64,4), (4,64,4), (16,64,4), (32,64,4)))
  slow_str = "Slow Slave " if slow else ""
  if single_stage:
    plot_parameter_combination(all_parameter_combinations, list(single_stage_experiments), slow_str + "Single Stage Geometric mean", bandwidth, slow)
  else:
    plot_parameter_combination(all_parameter_combinations, all_experiments, slow_str + "Geometric mean", bandwidth, slow)


plot_geomean_of_all(bandwidth=1, slow=0, single_stage=0)
plot_geomean_of_all(bandwidth=0, slow=0, single_stage=0)
plot_geomean_of_high_load(bandwidth=1, slow=0, single_stage=0)
plot_geomean_of_high_load(bandwidth=0, slow=0, single_stage=0)
plot_all_parameter_combinations(bandwidth=1,slow=0, single_stage=0)
plot_all_parameter_combinations(bandwidth=0,slow=0, single_stage=0)
# slow slave experiments
plot_geomean_of_all(bandwidth=1, slow=1, single_stage=0)
plot_geomean_of_all(bandwidth=0, slow=1, single_stage=0)
plot_geomean_of_high_load(bandwidth=1, slow=1, single_stage=0)
plot_geomean_of_high_load(bandwidth=0, slow=1, single_stage=0)
plot_all_parameter_combinations(bandwidth=1,slow=1, single_stage=0)
plot_all_parameter_combinations(bandwidth=0,slow=1, single_stage=0)
# single stage
plot_geomean_of_all(bandwidth=1, slow=0, single_stage=1)
plot_geomean_of_all(bandwidth=0, slow=0, single_stage=1)
plot_geomean_of_high_load(bandwidth=1, slow=0, single_stage=1)
plot_geomean_of_high_load(bandwidth=0, slow=0, single_stage=1)
plot_all_parameter_combinations(bandwidth=1, slow=0, single_stage=1)
plot_all_parameter_combinations(bandwidth=0, slow=0, single_stage=1)


In [ ]:
def plot_single_experiment_bar_plot(parameter_combination, routers,routers_legend, title, bounds=(0.9,1.1), bandwidth=1, slow=0):
  experiment_addon = "slowSlave" if slow else ""
  data_exp_list = []
  for experiment in routers:
    try:
      data_exp, traffic_types, parameter_combinations = get_data(experiment_addon + experiment)
      if parameter_combination not in parameter_combinations:
        print(f"Parameter combination {parameter_combination} not found in {experiment_addon + experiment}")
        continue
      narrow_latency_comparison = data_exp[:,parameter_combinations.index(parameter_combination),1,0]/ data_exp[:,parameter_combinations.index(parameter_combination),0,0]
      wide_bw_comparison = data_exp[:,parameter_combinations.index(parameter_combination),1,1]/ data_exp[:,parameter_combinations.index(parameter_combination),0,1]
      data_exp = wide_bw_comparison if bandwidth else narrow_latency_comparison
      data_exp_list.append(data_exp)
    except FileNotFoundError:
        print(f"File not found: output/dma_experiments_{experiment_addon + experiment}.txt")
        continue
  fig, ax = plt.subplots()
  x = np.arange(len(traffic_types))

  width = 1/(len(data_exp_list)+1)
  for i, data_exp in enumerate(data_exp_list[::-1]):
    ax.bar(x + (i - len(data_exp_list)/2)*width, data_exp_list[i] -0.8, width , label=routers_legend[i], bottom=0.8)

  ax.set_ylabel('$BW_{vc}\ /\ BW_{old}$' if bandwidth else '$Lat_{vc}\ /\ Lat_{old}$')
  ax.set_title(f'{title}')
  ax.set_ylim(bounds[0],bounds[1])
  ax.set_xticks(x)
  traffic_types = map(lambda x: x.replace("_", " "), traffic_types)
  ax.set_xticklabels(traffic_types)
  ax.yaxis.grid(True, linestyle='--', which='major',
                   color='grey', alpha=.25)
  plt.axhline(1, color='black', linewidth=0.5)
    # Rotate the tick labels and set their alignment.
  plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
          rotation_mode="anchor")
  ax.legend()
  fig.set_size_inches(10, 5)
  plt.savefig(f"util/plots/barplot_{routers}_{bandwidth}_{slow}.pdf", bbox_inches='tight')
  plt.show()

# All plots are without hbm and single dest boundary
plot_single_experiment_bar_plot((4,4,16), ("VCd2", "VCd3"),("Buffer depth 2", "Buffer depth 3"), "Low load with bursts: Normalized wide bandwidth: Different buffer depths", bandwidth=1, slow=0)
plot_single_experiment_bar_plot((4,4,16), ("VCd2Whd3VCL1", "VCd3"),("Buffer depth 2-3, local port optimized", "Buffer depth 3"), "Low load with bursts: Normalized wide bandwidth: Optimized router", bandwidth=1, slow=0)
plot_single_experiment_bar_plot((4,4,16), ("SingleVCd2", "SingleVCd3"),("Buffer depth 2, no VC", "Buffer depth 3, no VC"), "Low load with bursts: Normalized wide bandwidth: Different buffer depths without virtual channels", bounds = (0.8,1.1), bandwidth=1, slow=0)
plot_single_experiment_bar_plot((4,4,16), ("VCd2", "VCd2NoCreditShortcut"),("Buffer depth 2", "Low load with bursts: Buffer depth 2, no credit shortcut"), "Low load with bursts: Normalized wide bandwidth: Credit shortcut with buffer depth 2", bandwidth=1, slow=0)
plot_single_experiment_bar_plot((4,4,16), ("VCd3", "VCd3NoCreditShortcut"),("Buffer depth 3", "Buffer depth 3, no credit shortcut"), "Low load with bursts: Normalized wide bandwidth: Credit shortcut with buffer depth 3", bandwidth=1, slow=0)
plot_single_experiment_bar_plot((4,4,16), ("SingleStageVCd2VCL1", "SingleStageVCd3VCL1"),("Single Stage Buffer depth 2, optimized local port", "Single Stage Buffer depth 3, optimized local port"), "Low load with bursts: Normalized wide bandwidth: Single stage routers with different buffer depth", bounds=(0.9, 1.2), bandwidth=1, slow=0)
